In [1]:
import gc
import os
import json
import torch
import pickle
import pandas as pd
import transformers
from tqdm import tqdm
from openai import AzureOpenAI
from hallucination_editor import *

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
def load_api_key(key, file_path='api_key.json'):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data[key]

def get_response(model, tok, messages, max_new_tokens=1):
    terminators = [tok.eos_token_id, tok.convert_tokens_to_ids("<|eot_id|>")]
    msg_tokenized = tok.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt').to(model.device)
    output_ids = model.generate(msg_tokenized, max_new_tokens=max_new_tokens, eos_token_id=terminators, do_sample=False, pad_token_id=tok.eos_token_id)
    return tok.decode(output_ids[0][msg_tokenized.shape[-1]:], skip_special_tokens=True).replace('\n', ' ').strip().rstrip('.')

def get_gpt_response(client, system_msg, prompt, model='gpt-4o', temperature=0):
    raw_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"},
        temperature=temperature
    )
    return raw_response

device = 'cuda:0'
topic_ls = ['places_city', 'places_country', 'places_landmark', 'entertainment_song', 'entertainment_anime', 'entertainment_music_genre', 
             'art_literary', 'art_sculpture', 'health_treatment', 'health_medication', 'health_disease', 'human_politician', 'human_writer', 'human_actor', 
             'event_sport', 'event_history', 'event_film']
model_ls = ['meta-llama/Meta-Llama-3.1-8B-Instruct', 'mistralai/Mistral-7B-Instruct-v0.3', 'google/gemma-2-9b-it', 
            'lmsys/vicuna-7b-v1.5', 'chavinlo/alpaca-native', 'meta-llama/Meta-Llama-3-8B-Instruct']
model_id = model_ls[0]
domain_topic_name = topic_ls[2]
model_id_format = model_id.split('/')[-1].replace('-', '_').lower()
print(f'model_id: {model_id}, current topic: {domain_topic_name}')
system_msg_qa, system_msg_eval#, model_eval

model_id: meta-llama/Meta-Llama-3.1-8B-Instruct, current topic: places_landmark


('Always respond to the input question concisely with a short phrase or a single-word answer. Do not repeat the question or provide any explanation.',
 "Given two texts, labeled as Text 1 and Text 2, output '1' if they match each other semantically; otherwise, output '0'. Do not repeat the question or provide any explanation.")

## Generate Questions from knowledge graph triplets

To avoid question with multiple answers, check and remove (node, relation) with duplicates as shown below:\
{"subject": "Thailand", "relation": "diplomatic relation", "object": "Russia"}\
{"subject": "Thailand", "relation": "diplomatic relation", "object": "Brunei"}

In [2]:
with open(f'../data/graph/places_country.gpickle', 'rb') as f:
    directed_graph = pickle.load(f)
for node in directed_graph.nodes():
    for _, target, edge_data in directed_graph.edges(node, data=True):
        print(f"out_edges, node: {node}, relation: {edge_data['label']}, target: {target}")

out_edges, node: Thailand, relation: diplomatic relation, target: North Korea
out_edges, node: Thailand, relation: contains the administrative territorial entity, target: Si Sa Ket
out_edges, node: Thailand, relation: official symbol, target: Asian elephant
out_edges, node: Thailand, relation: member of, target: UNESCO
out_edges, node: Thailand, relation: head of government, target: Prayut Chan-o-cha
out_edges, node: Thailand, relation: shares border with, target: Laos
out_edges, node: Thailand, relation: contains the administrative territorial entity, target: Nonthaburi
out_edges, node: Thailand, relation: member of, target: Interpol
out_edges, node: Thailand, relation: contains the administrative territorial entity, target: Chumphon
out_edges, node: Thailand, relation: diplomatic relation, target: South Korea
out_edges, node: Thailand, relation: contains the administrative territorial entity, target: Maha Sarakham
out_edges, node: Thailand, relation: diplomatic relation, target: Saud

In [13]:
remove_relation = ["topic's main category", "topic's main template", "described by source", "Commons category", "on focus list of Wikimedia project"]

def generate_triplets(topic):
    with open(f'../data/graph/{topic}.gpickle', 'rb') as f:
        directed_graph = pickle.load(f)

    remove_pairs = set()
    triplets, all_pairs = [], []
    for node in directed_graph.nodes():
        for _, target, edge_data in directed_graph.out_edges(node, data=True):
            relation = edge_data['label']
            if (node, relation) in all_pairs:
                remove_pairs.add((node, relation))
            else:
                all_pairs.append((node, relation))
                print(f"out_edges, node: {node}, relation: {relation}, target: {target}")

        for _, target_in, edge_data in directed_graph.in_edges(node, data=True):
            relation = edge_data['label']
            if (node, relation) in all_pairs:
                remove_pairs.add((node, relation))
            else:
                all_pairs.append((node, relation))
                print(f"in_edges, node: {node}, relation: {relation}, target: {target_in}")
        

    for node in directed_graph.nodes():
        for _, target, edge_data in directed_graph.out_edges(node, data=True):
            relation = edge_data['label']
            if (node, relation) not in remove_pairs:
                triplets.append({"subject": node, "relation": relation, "object": target})

        for _, target, edge_data in directed_graph.in_edges(node, data=True):
            relation = edge_data['label']
            if (node, relation) not in remove_pairs:
                triplets.append({"subject": node, "relation": relation, "object": target})

    df_triplets = pd.DataFrame(triplets)
    # df_triplets.to_csv(f"../data/questions/triplets/tmp_{topic}.csv", index=False, encoding='utf-8')
    print(f"Saving to ../data/questions/triplets/{topic}.csv. Triplets size: {len(df_triplets)}")
    # return df_triplets


# for filename in os.listdir(directory):
#     if filename.endswith('.gpickle'):
#         df_triplets = generate_triplets(filename.split('.')[0])
# generate_triplets('entertainment_anime')
generate_triplets('places_country')

out_edges, node: Thailand, relation: diplomatic relation, target: North Korea
out_edges, node: Thailand, relation: contains the administrative territorial entity, target: Si Sa Ket
out_edges, node: Thailand, relation: official symbol, target: Asian elephant
out_edges, node: Thailand, relation: member of, target: UNESCO
out_edges, node: Thailand, relation: head of government, target: Prayut Chan-o-cha
out_edges, node: Thailand, relation: shares border with, target: Laos
out_edges, node: Thailand, relation: part of, target: Southeast Asia
out_edges, node: Thailand, relation: described by source, target: Encyclopædia Britannica 11th edition
out_edges, node: Thailand, relation: official language, target: Thai
out_edges, node: Thailand, relation: instance of, target: constitutional monarchy
out_edges, node: Thailand, relation: religion or worldview, target: Christianity
out_edges, node: Thailand, relation: continent, target: Asia
out_edges, node: Thailand, relation: head of state, target: V

In [57]:
x = set(remove_pairs_in).intersection(set(remove_pairs_out))
len(remove_pairs_in), len(remove_pairs_out), len(set(remove_pairs_in)), len(set(remove_pairs_out)), len(x)

(958, 684, 214, 154, 0)

In [ ]:
system_msg_gen_question = """Given a subject and a relation, generate a question that asks about the subject. \
Output the result in JSON format as demonstrated in the following example.
Example input: subject: Interstellar, relation: director
Example output: {"question": "Who is the director of Interstellar?"}"""
client = AzureOpenAI(api_key=load_api_key('api_key_n_central_us'), api_version='2023-05-15', azure_endpoint="https://n-central-us.openai.azure.com/")
# client = AzureOpenAI(api_key=load_api_key('api_key_east_us'), api_version='2023-05-15', azure_endpoint="https://east-us-one.openai.azure.com/")
questions = []

def generate_questions(df):
    for i in df.index[:]:
        subject, relation = df.loc[i, 'subject'], df.loc[i, 'relation']
        prompt_gen_question = f"subject: {subject}, relation: {relation}"
        raw_response = get_gpt_response(client, system_msg_gen_question, prompt_gen_question)
        json_obj = json.loads(raw_response.choices[0].message.content)
        questions.append(json_obj['paraphrased_question'])
        print(json_obj)

generate_questions(df_triplets)

In [10]:
tok_qa = transformers.AutoTokenizer.from_pretrained(model_id)
model_qa = transformers.AutoModelForCausalLM.from_pretrained(model_id).to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# Check if subject == object
# for domain_topic_tmp in ['entertainment_anime', 'places_country', 'event_sport']:
for filename in os.listdir("../data/questions/unfiltered"):
    # print(filename)
    df = pd.read_csv(f"../data/questions/unfiltered/{filename}")
    if len(df[df['subject'] == df['object']]) > 0:
        print(f"In {filename}, there are {len(df[df['subject'] == df['object']])} questions where subject == object")
        # df = df[df['subject'] != df['object']]
    if len(df[df['label'] != df['object']]) > 0:
        print(f"In {filename}, there are {len(df[df['label'] != df['object']])} questions where label != object")

In human_writer.csv, there are 2 questions where subject == object


In [20]:
domain_topic_tmp = 'event_sport'
df = pd.read_csv(f"../data/questions/unfiltered/{domain_topic_tmp}_questions.csv")
print(f"Remove has part(s): {len(df[df['relation']=='has part(s)'])}")
print(f"Remove sponsor: {len(df[df['relation']=='sponsor'])}")
df = df[df['relation'] != 'sponsor']
df = df[df['relation'] != 'has part(s)']
df.to_csv(f"../data/questions/unfiltered/{domain_topic_tmp}_questions_2.csv", index=False)

Remove has part(s): 12
Remove sponsor: 7


In [22]:
domain_topic_tmp = 'places_country'
df = pd.read_csv(f"../data/questions/unfiltered/{domain_topic_tmp}_questions.csv")
print(f"Remove: {len(df[df['relation']=='twinned administrative body'])}")
# df.to_csv(f"../data/questions/unfiltered/{domain_topic_tmp}_questions_2.csv", index=False)

Remove has part(s): 9


## QA

In [16]:
# Get response for questions
print(model_id_format)
# df_all_topics = pd.DataFrame()

# for domain_topic_tmp in topic_ls:
# for domain_topic_tmp in ['entertainment_anime']:  # entertainment_anime places_country
#     print(f"file name: {domain_topic_tmp}")
#     df = pd.read_csv(f"../data/questions/unfiltered/{domain_topic_tmp}_questions.csv")
for filename in os.listdir("../data/questions/unfiltered")[1:]:
    df = pd.read_csv(f"../data/questions/unfiltered/{filename}")
    ls_output = []
    
    for i in tqdm(df.index[:]):
        question = df.loc[i, 'question']
        # user_msg_qa = Wh_content + "\nQuestion:" + question  # places_landmark_old.csv
        # user_msg_qa = f'Question: {question} Answer:'
        user_msg_qa = f'{question}'
        if model_id_format == 'gemma_2_9b_it':  # System role not supported for gemma
            messages_qa = [{"role": "user", "content": system_msg_qa+' '+user_msg_qa}]
        else:
            messages_qa = [{"role": "system", "content": system_msg_qa}, {"role": "user", "content": user_msg_qa}]
        output_qa = get_response(model_qa, tok_qa, messages_qa, max_new_tokens=16)
        # if '\n' in output_qa:
            # print(f"Question: {question} Label: {label} | Prediction: {repr(output_qa)}")
        # output_qa = output_qa.replace('\n', ' ').strip().rstrip('.')  # remove trailing period for llama output
        ls_output.append(output_qa)
    
    df['topic'] = filename.replace('.csv', '')
    df[f"output_{model_id_format}"] = ls_output
    df[['topic', 'subject', 'relation', 'object', 'question', f'output_{model_id_format}']].to_csv(f"../data/questions/unfiltered/{filename}", index=False)
    print(filename)
    # df_all_topics = pd.concat([df_all_topics, df], axis=0)
    # print("df_all_topics.shape:", df_all_topics.shape)
# df_all_topics = df_all_topics[['topic', 'type', 'subject', 'relation', 'object', 'question', 'label', f'output_{model_id_format}']]
# df_all_topics.to_csv(f"../data/questions/wh_only/all_topics_{model_id_format}.csv", index=False)
# del model_qa
# gc.collect()
# torch.cuda.empty_cache()

meta_llama_3.1_8b_instruct


  0%|          | 0/300 [00:00<?, ?it/s]/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 300/300 [02:05<00:00,  2.39it/s]


entertainment_music_genre.csv


  0%|          | 0/1935 [00:00<?, ?it/s]/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
 71%|███████   | 1373/1935 [10:34<04:10,  2.24it/s]

## Evaluate to filter hallucination

In [24]:
def evaluate_responses(model_eval, tok_eval, df_wh, system_msg_eval, user_msg_eval_template="Text 1: {label} \nText 2: {output_qa}"):
    wh_count = 0
    wh_correct = 0
    for i in df_wh.index[:]:
        label = df_wh.loc[i, 'object']
        output_qa = df_wh.loc[i, f"output_{model_id_format}"]
        eval_res = 0
        wh_count += 1 

        if output_qa.lower() in label.lower() or label.lower() in output_qa.lower():  # Rule-based fuzzy match
            wh_correct += 1
            eval_res = 1
            if output_qa.lower() == label.lower():
                print(f"Label: {label:<35} Prediction: {output_qa:<35} Evaluation: Exact Match")
            else:
                print(f"Label: {label:<35} Prediction: {output_qa:<35} Evaluation: Partial Match")
        else:
            user_msg_eval = user_msg_eval_template.format(label=label, output_qa=output_qa)
            messages_eval = [{"role": "system", "content": system_msg_eval}, {"role": "user", "content": user_msg_eval}]
            response_eval = get_response(model_eval, tok_eval, messages_eval)
            if response_eval != '0':
                print(f"Label: {label:<35} Prediction: {output_qa:<35} Evaluation: Semantic Match")
                wh_correct += 1
                eval_res = 1
                
        df_wh.loc[i, f"eval_{model_id_format}"] = eval_res
    print(f"\nHallucination ratio: {1 - wh_correct/wh_count} df_hallucination.shape: {df_wh[df_wh[f'eval_{model_id_format}']==0].shape}")
    return df_wh
print(system_msg_eval)

Given two texts, labeled as Text 1 and Text 2, output '1' if they match each other semantically; otherwise, output '0'. Do not repeat the question or provide any explanation.


In [25]:
domain_topic_name = 'entertainment_anime'
df_wh = pd.read_csv(f"../data/questions/unfiltered/{domain_topic_name}_questions.csv")
# df_wh = df_wh[df_wh.topic == domain_topic_name]
print(f"domain_topic_name: {domain_topic_name}, df_wh.shape: {df_wh.shape}")

# if not os.path.exists(f"../data/questions/wh_only/hallucination_only/{model_id_format}"):
#     os.makedirs(f"../data/questions/wh_only/hallucination_only/{model_id_format}")
df_wh = evaluate_responses(model_eval, tok_eval, df_wh, system_msg_eval)
df_hallu = df_wh[df_wh[f"eval_{model_id_format}"] == 0]
# df_hallu.to_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}.csv", index=False)
# df_wh[df_wh[f"eval_{model_id_format}"] == 1]
# The wh question accuracy of the language model is 0.

domain_topic_name: entertainment_anime, df_wh.shape: (276, 6)


/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/dat

Label: Japanese                            Prediction: Japanese                            Evaluation: Exact Match
Label: Japan                               Prediction: Japan                               Evaluation: Exact Match
Label: Japanese                            Prediction: Japanese                            Evaluation: Exact Match
Label: Japan                               Prediction: Japan                               Evaluation: Exact Match
Label: Japan                               Prediction: Japan                               Evaluation: Exact Match
Label: Japanese                            Prediction: Japanese                            Evaluation: Exact Match
Label: Japan                               Prediction: Japan                               Evaluation: Exact Match
Label: Japanese                            Prediction: Japanese                            Evaluation: Exact Match
Label: Japanese                            Prediction: Japanese                 

In [26]:
# Round 2: use system_msg_2 to filter case such as United Kingdom vs. England
system_msg_2 = "Given two texts, labeled as Text 1 and Text 2, output '1' if they if they have similar semantic meanings, are synonyms, \
or if one is a more specific or general version of the other; otherwise, output '0'. Do not repeat the question or provide any explanation."   

df_hallu = evaluate_responses(model_eval, tok_eval, df_hallu, system_msg_2)
df_hallu = df_hallu[df_hallu[f"eval_{model_id_format}"] == 0]
df_hallu.to_csv(f"../data/questions/hallucination/{model_id_format}_{domain_topic_name}.csv", index=False)

Label: video on demand                     Prediction: Streaming                           Evaluation: Semantic Match
Label: detective fiction                   Prediction: Mystery                             Evaluation: Semantic Match
Label: Macross Delta                       Prediction: Macross franchise                   Evaluation: Semantic Match
Label: video on demand                     Prediction: Digital distribution                Evaluation: Semantic Match

Hallucination ratio: 0.9775280898876404 df_hallucination.shape: (174, 7)


In [ ]:
Remove unknown

### Other evaluation less/more strict

In [ ]:
# system_msg_eval = """Given a question, a label, and a prediction, evaluate the correctness of the prediction compared to the label. \
# Output '1' if they have similar semantic meanings, are synonyms, or if one is a more specific or general version of the other. Otherwise, output '0'. \
# Only output the final evaluation as a single word. Do not repeat the question or provide an explanation."""

In [ ]:
domain_topic_name = 'places_landmark'
df_wh = pd.read_csv(f"../data/questions/wh_only/all_topics_{model_id_format}.csv")
df_wh = df_wh[df_wh.topic == domain_topic_name]
print(domain_topic_name, df_wh.shape)

system_msg_eval = """Given a label and a prediction, evaluate the correctness of the prediction compared to the label. \
Output '1' if they match each other semantically. Otherwise, output '0'. Do not repeat the question or provide an explanation."""
# output the final evaluation as a single word. Do not repeat the question or provide an explanation Only output '0' or '1'.

user_msg_eval_template = f"""label: {label} \nprediction: {output_qa}\n"""
df_wh = evaluate_responses(model_eval, tok_eval, df_wh, system_msg_eval, user_msg_eval_template)

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 
The wh question accuracy of the language model is 0.418


In [ ]:
domain_topic_name = 'places_landmark'
df_wh = pd.read_csv(f"../data/questions/wh_only/all_topics_{model_id_format}.csv")
df_wh = df_wh[df_wh.topic == domain_topic_name]

system_msg_eval = """Given a question, a label, and a prediction, evaluate the correctness of the prediction compared to the label. \
Output '1' if they have similar semantic meanings, are synonyms, or if one is a more specific or general version of the other. Otherwise, output '0'. \
Only output the final evaluation as a single word. Do not repeat the question or provide an explanation."""

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label, output_qa = df_wh.loc[i, 'question'], df_wh.loc[i, 'label'], df_wh.loc[i, f"output_{model_id_format}"]
    prompt_eval = f"""question: {question} \nlabel: {label} \nprediction: {output_qa}\n"""
    eval_res = 0
    wh_count += 1 

    if output_qa.lower() in label.lower() or label.lower() in output_qa.lower():  # Rule-basd fuzzy match
        wh_correct += 1
        eval_res = 1
    else:
        
        messages_eval = [{"role": "system", "content": system_msg_eval}, {"role": "user", "content": user_msg_eval}]
        response_eval = get_response(model_eval, tok_eval, messages_eval)
        if response_eval == '1':
            wh_correct += 1
            eval_res = 1
            
    df_wh.loc[i, f"eval_{model_id_format}"] = eval_res
    
print(f"The wh question accuracy of the language model is {wh_correct / wh_count}")
# if not os.path.exists(f"../data/questions/wh_only/hallucination_only/{model_id_format}"):
#     os.makedirs(f"../data/questions/wh_only/hallucination_only/{model_id_format}")
df_wh[df_wh[f"eval_{model_id_format}"] == 0].to_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}.csv", index=False)
df_wh[df_wh[f"eval_{model_id_format}"] == 1]
# The wh question accuracy of the language model is 0.656

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The wh question accuracy of the language model is 0.642


,topic,type,subject,relation,object,question,label,output_meta_llama_3.1_8b_instruct,eval_meta_llama_3.1_8b_instruct
2000,places_landmark,wh,Old Royal Naval College,architect,Christopher Wren,Who does Old Royal Naval College architect?,Christopher Wren,Christopher Wren.,1.0
2001,places_landmark,wh,Old Royal Naval College,country,United Kingdom,What is the country of Old Royal Naval College?,United Kingdom,United Kingdom.,1.0
2003,places_landmark,wh,Panathenaic Stadium,occupant,Hellenic Olympic Committee,What is the occupant of Panathenaic Stadium?,Hellenic Olympic Committee,Greek National Stadium.,1.0
2004,places_landmark,wh,Panathenaic Stadium,made from material,marble,What is the made from material of Panathenaic ...,marble,Pentelic marble.,1.0
2005,places_landmark,wh,Panathenaic Stadium,sponsor,Stavros Niarchos Foundation,What does Panathenaic Stadium sponsor?,Stavros Niarchos Foundation,Athletic events.,1.0
...,...,...,...,...,...,...,...,...,...
2492,places_landmark,wh,MUNCH,located in or next to body of water,Oslofjord,What is the located in or next to body of wate...,Oslofjord,Oslofjord.,1.0
2494,places_landmark,wh,MUNCH,replaces,Munch Museum,What does MUNCH replace?,Munch Museum,MUNCH replaces OMNISCIENT.,1.0
2496,places_landmark,wh,Bridget of Sweden,founded by,Vadstena Abbey,Which tourist attraction was founded by Bridge...,Vadstena Abbey,Birgitta Abbey.,1.0
2498,places_landmark,wh,St Mark's Campanile,has part(s),St Mark's Basilica,Which tourist attraction has part(s) St Mark's...,St Mark's Basilica,St. Mark's Square.,1.0


In [7]:
# May output other than 0 or 1
# system_msg_eval = """Given a question, a correct answer, and a prediction, evaluate if the prediction and the correct answer match semantically. \
# Output '1' if they have similar meanings, are synonyms, or if one is a more specific or general version of the other. Otherwise, output '0'."""

system_msg_eval = """Given a question, a correct answer, and a prediction, evaluate whether the prediction and the correct answer match semantically. \
Output '1' if they convey similar meanings, including when the prediction is more specific, more general, or a synonym of the correct answer. Otherwise, output '0'."""

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label, output = df_wh.loc[i, 'question'], df_wh.loc[i, 'label'], df_wh.loc[i, f"output_{model_id}"]
    prompt_eval = f"""The inputs are given as below: \nquestion: {question} \n\ncorrect answer: {label} \n\nprediction: {output}\n"""

    eval_res = 0
    wh_count += 1

    if output.lower() in label.lower() or label.lower() in output.lower():  # Rule-basd fuzzy match
        wh_correct += 1
        eval_res = 1
    else:
        messages = [{"role": "system", "content": system_msg_eval}, {"role": "user", "content": prompt_eval+" Only output '1' or '0'."}]
        msg_tokenized = tok.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt')
        output_ids = model_eval.generate(msg_tokenized.to(device_eval), max_new_tokens=1, eos_token_id=terminators, do_sample=False, pad_token_id=tok.eos_token_id)
        response_str = tok.decode(output_ids[0][msg_tokenized.shape[-1]:], skip_special_tokens=True)

        if response_str == '1':
            wh_correct += 1
            eval_res = 1
            
    df_wh.loc[i, f"eval_{model_id_format}"] = eval_res
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh[df_wh[f"eval_{model_id_format}"] == 0].to_csv(f"../data/questions/wh_only/hallucination_only/{domain_topic_name}_{model_id_format}_eval.csv", index=False)
df_wh[df_wh[f"eval_{model_id_format}"] == 1]

/home/baix/anaconda3/envs/env24may/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/baix/anaconda3/envs/env24may/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


[GPT] The wh question accuracy of the language model is 0.732


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta_llama_3_8b_instruct
0,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
2,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
3,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
4,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,1.0
497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,1.0


In [16]:
from sentence_transformers import SentenceTransformer, util

ls_label = df_wh.label.tolist()

wh_count = 0
wh_correct = 0
model_name = 'paraphrase-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    flag = False

    wh_count += 1
    embeddings = model.encode([label, output])
    similarity_score = util.cos_sim(embeddings[0], embeddings[1])
    threshold = 0.6
    if similarity_score >= threshold:
        wh_correct += 1
        flag = True
        
wh_acc_dict = {"wh_accuracy": wh_correct/wh_count}
print(f"[Sentence Transformer] The wh question accuracy of the language model is {wh_correct / wh_count}")

[Sentence Transformer] The wh question accuracy of the language model is 0.418


In [18]:
llm_name = 'gpt-35-turbo'

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    
    info = "Question: "+question+ "\nModel Answer: "+label + "\nAI model generated answer: " + output \
        + "\n. The above question's topic is " + domain_topic_name  + ". "
    string = """\n Please evaluate the correctness of the AI model's answer compared to the model answer. 
        Consider the following criteria and provide your judgment:
        If the AI's answer is a more specific version of the model answer, please respond with: "Correct"
        If the AI's answer is a more general version of the model answer, please respond with: "Correct".
        If the AI's answer is a closely related to the model answer, please respond with: "Correct".
        If the AI's answer and the model answer are entirely different entities with no direct relationship, please respond with: "Incorrect".
        """

    wh_count += 1
    raw_response = client.chat.completions.create(
        model=llm_name, 
        messages=[{"role": "system", "content": ""}, {"role": "user", "content": info + string}], 
        temperature=0
    )
    response_str = raw_response.choices[0].message.content.strip().replace('\n\n\n', '\n\n')
    # response_str

    if response_str and response_str.rstrip('.') == "Correct":
        wh_correct += 1
        df_wh.loc[i, f"eval_{model_id}"] = 1
    else:
        df_wh.loc[i, f"eval_{model_id}"] = 0
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[GPT] The wh question accuracy of the language model is 0.694


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1003,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
1495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
1496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,0.0
1497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
1498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,1.0


In [19]:
llm_name = 'gpt-35-turbo'
topic = 'health_medication'

system_msg_eval = "Given two texts, labeled as Text 1 and Text 2, output '1' if they match each other semantically, and output '0' if they do not."

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    
    # info = "Question: "+question+ "\nModel Answer: "+label + "\nAI model generated answer: " + output \
    #     + "\n. The above question's topic is " + topic  + ". "
    prompt_eval = f"""The input texts are given as below: \nText 1: {label} \n\nText 2: {output}\n"""
    
    wh_count += 1
    raw_response = client.chat.completions.create(
        model=llm_name, 
        messages=[{"role": "system", "content": system_msg_eval}, {"role": "user", "content": prompt_eval}], 
        temperature=0
    )
    response_str = raw_response.choices[0].message.content.strip().replace('\n\n\n', '\n\n')

    if str(response_str) == '1':
        wh_correct += 1
        df_wh.loc[i, f"eval_{model_id}"] = 1
    else:
        df_wh.loc[i, f"eval_{model_id}"] = 0
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh

[GPT] The wh question accuracy of the language model is 0.252


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1003,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
1495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
1496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,0.0
1497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
1498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,0.0


In [21]:
df_wh[df_wh[f"eval_{model_id}"]==1]

,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
1012,wh,John Rylands Library,country,United Kingdom,What is the country of John Rylands Library?,United Kingdom,United Kingdom,1.0
...,...,...,...,...,...,...,...,...
1474,wh,Jōshin'etsu-kōgen National Park,located in protected area,Shiga Highlands,Which tourist attraction's located in protecte...,Shiga Highlands,Shiga Kogen,1.0
1475,wh,Mount Kilimanjaro,country,Tanzania,What is the country of Mount Kilimanjaro?,Tanzania,Tanzania,1.0
1484,wh,Night Safari,country,Singapore,What is the country of Night Safari?,Singapore,Singapore,1.0
1489,wh,St Paul's Cathedral,architect,Christopher Wren,Who does St Paul's Cathedral architect?,Christopher Wren,Christopher Wren,1.0


In [20]:
system_msg_eval = """Given a question, a correct answer, and a prediction, evaluate whether the prediction is semantically equivalent to the correct answer. \
Output '1' if they are semantically equivalent, otherwise output '0'."""

wh_count = 0
wh_correct = 0
for i in df_wh.index[:]:
    question, label = df_wh.loc[i, 'question'], df_wh.loc[i, 'label']
    output = df_wh.loc[i, f"output_{model_id}"]
    
        # + "\n. The above question's topic is " + topic  + ". "
    prompt_eval = f"""The inputs are given as below: \nquestion: {question} \n\ncorrect answer: {label} \n\nprediction: {output}\n"""
    
    wh_count += 1
    raw_response = client.chat.completions.create(
        model=llm_name, 
        messages=[{"role": "system", "content": system_msg_eval}, {"role": "user", "content": prompt_eval}], 
        temperature=0
    )
    response_str = raw_response.choices[0].message.content.strip().replace('\n\n\n', '\n\n')

    if str(response_str) == '1':
        wh_correct += 1
        df_wh.loc[i, f"eval_{model_id}"] = 1
    else:
        df_wh.loc[i, f"eval_{model_id}"] = 0
print(f"[GPT] The wh question accuracy of the language model is {wh_correct / wh_count}")
df_wh

[GPT] The wh question accuracy of the language model is 0.278


,type,subject,relation,object,question,label,output_meta-llama/Meta-Llama-3-8B-Instruct,eval_meta-llama/Meta-Llama-3-8B-Instruct
1000,wh,Ontario,located in the administrative territorial entity,Niagara Falls,Which tourist attraction's located in the admi...,Niagara Falls,Niagara Falls,1.0
1001,wh,Alexandrov Kremlin,country,Russia,What is the country of Alexandrov Kremlin?,Russia,Russia,1.0
1002,wh,Alexandrov Kremlin,located in the administrative territorial entity,Alexandrov,Who is the located in the administrative terri...,Alexandrov,Alexandrov,1.0
1003,wh,Bukit Panjang,located in the administrative territorial entity,Bukit Timah Nature Reserve,Which tourist attraction's located in the admi...,Bukit Timah Nature Reserve,Haw Par Villa,0.0
1004,wh,Kastelholm Castle,country,Finland,What is the country of Kastelholm Castle?,Finland,Finland,1.0
...,...,...,...,...,...,...,...,...
1495,wh,Thornton Tomasetti,structural engineer,Petronas Towers,Which tourist attraction's structural engineer...,Petronas Towers,One World Trade,0.0
1496,wh,Charles II of England,occupant,Windsor Castle,Which tourist attraction's occupant is Charles...,Windsor Castle,Westminster Abbey,0.0
1497,wh,Charles II of England,founded by,Royal Observatory,Which tourist attraction was founded by Charle...,Royal Observatory,St. Paul's,0.0
1498,wh,Gateway Arch,located in protected area,Gateway Arch National Park,What is the located in protected area of Gatew...,Gateway Arch National Park,Jefferson National Park,0.0


### Generate yes_no, portability, and locality questions

2-stage filter

In [34]:
df_wh_hallu = pd.read_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}_{domain_topic_name}.csv")
df_wh_hallu = df_wh_hallu[df_wh_hallu.topic==domain_topic_name]
print(len(df_wh_hallu), domain_topic_name)
df_wh_hallu.head(2)

204 places_country


,topic,type,subject,relation,object,question,label,output_meta_llama_3.1_8b_instruct,eval_meta_llama_3.1_8b_instruct
0,places_country,wh,Botswana,separated from,United Kingdom,What was Botswana separated from?,United Kingdom,Bechuanaland,0.0
1,places_country,wh,Botswana,country,Botswana,What is the country of Botswana?,Botswana,Nation,0.0


In [36]:
system_msg_gen_q = """Given a subject, relation, object, and a question asking about the object, generate following questions of different types:
1. Paraphrased question: Generate a paraphrased question asking about the same subject. The answer should still be the object.
2. Yes question: Rewrite the original question as a yes/no question by including the object, so that the correct answer should be "Yes".
3. No question: Rewrite the original question as a yes/no question by including the wrong answer, so that the correct answer should be "No".
4. Multiple-Choice Question: Create a question with four answer options and the ground truth (correct answer). Include the correct answer (object), \
one provided wrong answer, and two additional distractors. Randomly assign these options to choices A, B, C, and D. Use the following JSON format: \
{"question": "...", "options": {"A": "...", "B": "...", "C": "...", "D": "..."}, "ground_truth": "Only provide the correct answer as a letter (A, B, C, or D)"}
5. Locality question: Generate a locality question related to a well-known attribute of the same subject. The attribute should be unrelated to the object and relation from the original question.
6. Reversed relation question: Generate a question by reversing the subject and object in the original question so that the answer should be the original subject. 
Output in JSON format with the following keys: "paraphrased_question", "yes_question", "no_question", "multiple_choice_question", "locality_question", and "reversed_relation_question". \
"""
# Portability question: Create a portability question by replacing the subject in the original question with an alias or synonym. 


paraphrased_questions, yes_questions, no_questions, multiple_choice_questions, locality_questions, reversed_relation_questions = ([] for _ in range(6))


def generate_questions(df_wh_hallu):
    for i in df_wh_hallu.index[:]:
        subject, relation, object, question = df_wh_hallu.loc[i, 'subject'], df_wh_hallu.loc[i, 'relation'], df_wh_hallu.loc[i, 'object'], df_wh_hallu.loc[i, 'question']
        pre_edit_ans = df_wh_hallu.loc[i, f'output_{model_id_format}']
        prompt_gen_q = f"subject: {subject}, relation: {relation}, object: {object}, question: {question}, wrong answer: {pre_edit_ans}"
        raw_response = client.chat.completions.create(
            model='gpt-4o',  # -mini
            messages=[{"role": "system", "content": system_msg_gen_q}, {"role": "user", "content": prompt_gen_q}], 
            response_format={"type": "json_object"}, 
            temperature=0
        )
        json_obj = json.loads(raw_response.choices[0].message.content)
        # print(f'\nOriginal Question: {question} Object: {object}')
        print(json_obj)
        paraphrased_questions.append(json_obj['paraphrased_question'])
        yes_questions.append(json_obj['yes_question'])
        no_questions.append(json_obj['no_question'])
        multiple_choice_questions.append(json_obj['multiple_choice_question'])
        locality_questions.append(json_obj['locality_question'])
        reversed_relation_questions.append(json_obj['reversed_relation_question'])

generate_questions(df_wh_hallu)

{'paraphrased_question': 'From which country did Botswana gain its independence?', 'yes_question': 'Was Botswana separated from the United Kingdom?', 'no_question': 'Was Botswana separated from Bechuanaland?', 'multiple_choice_question': {'question': 'What was Botswana separated from?', 'options': {'A': 'United Kingdom', 'B': 'Bechuanaland', 'C': 'France', 'D': 'Germany'}, 'ground_truth': 'A'}, 'locality_question': 'What is the capital city of Botswana?', 'reversed_relation_question': 'Which country was separated from the United Kingdom?'}
{'paraphrased_question': 'Which country is Botswana?', 'yes_question': 'Is Botswana the country of Botswana?', 'no_question': 'Is Nation the country of Botswana?', 'multiple_choice_question': {'question': 'What is the country of Botswana?', 'options': {'A': 'Botswana', 'B': 'Nation', 'C': 'Zimbabwe', 'D': 'Namibia'}, 'ground_truth': 'A'}, 'locality_question': 'What is the capital city of Botswana?', 'reversed_relation_question': 'Which country is Bot

In [37]:
# df_wh_hallu = df_wh_hallu.iloc[:]
print(f"Before df_wh_hallu.shape: {df_wh_hallu.shape}")
df_wh_hallu['paraphrased_question'] = paraphrased_questions
df_wh_hallu['yes_question'] = yes_questions
df_wh_hallu['no_question'] = no_questions
df_wh_hallu['multiple_choice_question'] = multiple_choice_questions
df_wh_hallu['locality_question'] = locality_questions
df_wh_hallu['reversed_relation_question'] = reversed_relation_questions
print(f"After df_wh_hallu.shape: {df_wh_hallu.shape}")
df_wh_hallu.to_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}_{domain_topic_name}.csv", index=False)

Before df_wh_hallu.shape: (204, 9)
After df_wh_hallu.shape: (204, 15)


In [38]:
client = AzureOpenAI(api_key=load_api_key('api_key_n_central_us'), api_version='2023-05-15', azure_endpoint="https://n-central-us.openai.azure.com/")
system_msg_gen_q = """ 
Given a subject and a relation, your task is to create 2-hop, 3-hop, 4-hop, 5-hop, and 6-hop questions, along with their correct answers. \
Always use the provided subject and relation to create multi-hop questions, and avoid including any correct answers from other multi-hop questions. \
Output in JSON format. Below is an example:

Example input: 
subject: Amazon, relation: founder

Example output: 
{
    "2hop_question": "Who is the spouse of the Amazon founder?",
    "2hop_answer": "MacKenzie Scott",
    "3hop_question": "Which university did the spouse of the Amazon founder attend for their undergraduate studies?",
    "3hop_answer": "Princeton University",
    "4hop_question": "In which city is the university that the spouse of the Amazon founder attended located?",
    "4hop_answer": "Princeton",
    "5hop_question": "In which state is the city located where the university that the spouse of the Amazon founder attended is situated?",
    "5hop_answer": "New Jersey"
    "6hop_question": "In which country is the state located where the city is situated that contains the university the spouse of the Amazon founder attended?",
    "6hop_answer": "United States"
}
"""


ls_2hop_q, ls_2hop_a, ls_3hop_q, ls_3hop_a, ls_4hop_q, ls_4hop_a, ls_5hop_q, ls_5hop_a, ls_6hop_q, ls_6hop_a = ([] for _ in range(10))

def generate_questions(df_wh_hallu):
    for i in df_wh_hallu.index[:]:
        subject, relation, object, question = df_wh_hallu.loc[i, 'subject'], df_wh_hallu.loc[i, 'relation'], df_wh_hallu.loc[i, 'object'], df_wh_hallu.loc[i, 'question']
        prompt_gen_q = f"subject: {subject}, relation: {relation}"
        raw_response = client.chat.completions.create(
            model='gpt-4o',  # 'gpt-4o-mini'
            messages=[{"role": "system", "content": system_msg_gen_q}, {"role": "user", "content": prompt_gen_q}], 
            response_format={"type": "json_object"}, 
            temperature=0
        )
        raw_str = raw_response.choices[0].message.content
        json_obj = json.loads(raw_str)
        print(f"subject: {subject}, relation: {relation}, {json_obj}")
        ls_2hop_q.append(json_obj['2hop_question'])
        ls_2hop_a.append(json_obj['2hop_answer'])
        ls_3hop_q.append(json_obj['3hop_question'])
        ls_3hop_a.append(json_obj['3hop_answer'])
        ls_4hop_q.append(json_obj['4hop_question'])
        ls_4hop_a.append(json_obj['4hop_answer'])
        ls_5hop_q.append(json_obj['5hop_question'])
        ls_5hop_a.append(json_obj['5hop_answer'])
        ls_6hop_q.append(json_obj['6hop_question'])
        ls_6hop_a.append(json_obj['6hop_answer'])

generate_questions(df_wh_hallu)

subject: Botswana, relation: separated from, {'2hop_question': 'Which country is Botswana separated from?', '2hop_answer': 'South Africa', '3hop_question': 'What is the capital city of the country that Botswana is separated from?', '3hop_answer': 'Pretoria', '4hop_question': 'What is the official language spoken in the capital city of the country that Botswana is separated from?', '4hop_answer': 'Afrikaans', '5hop_question': 'What is the population of the country where the official language spoken in its capital city is Afrikaans?', '5hop_answer': 'Approximately 60 million', '6hop_question': 'What is the currency used in the country with a population of approximately 60 million where the official language of its capital city is Afrikaans?', '6hop_answer': 'South African Rand'}
subject: Botswana, relation: country, {'2hop_question': 'What is the capital city of the country Botswana?', '2hop_answer': 'Gaborone', '3hop_question': 'Which river flows near the capital city of Botswana?', '3h

In [39]:
print(f"Before df_wh_hallu.shape: {df_wh_hallu.shape}")
df_wh_hallu['question_2hop'] = ls_2hop_q
df_wh_hallu['answer_2hop'] = ls_2hop_a
df_wh_hallu['question_3hop'] = ls_3hop_q
df_wh_hallu['answer_3hop'] = ls_3hop_a
df_wh_hallu['question_4hop'] = ls_4hop_q
df_wh_hallu['answer_4hop'] = ls_4hop_a
df_wh_hallu['question_5hop'] = ls_5hop_q
df_wh_hallu['answer_5hop'] = ls_5hop_a
df_wh_hallu['question_6hop'] = ls_6hop_q
df_wh_hallu['answer_6hop'] = ls_6hop_a
print(f"After df_wh_hallu.shape: {df_wh_hallu.shape}")
df_wh_hallu.to_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}_{domain_topic_name}.csv", index=False)

Before df_wh_hallu.shape: (204, 15)
After df_wh_hallu.shape: (204, 25)


## Editing

In [12]:
import json
from hallucination_editor import BaseEditor
from easyeditor import FTHyperParams, IKEHyperParams, ROMEHyperParams, MEMITHyperParams

df = pd.read_csv(f"../data/questions/wh_only/hallucination_only/{model_id_format}.csv")
# df = pd.read_csv(f"../data/questions/wh_only/hallucination_only/mistral_7b_instruct_v0.3.csv")
df.shape

/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

(179, 9)

In [13]:
n = 50#len(df)
targets = df['label'].tolist()[:n]
subjects = df['subject'].tolist()[:n]
questions = df['question'].tolist()[:n]
# paraphrased_questions = df['paraphrased_questions'].tolist()[:n]
# portability_questions = df['portability_questions'].tolist()[:n]
# portability_inputs = {'subject_aliasing': {'prompt': portability_questions, 'ground_truth': answers},}

hparams = ROMEHyperParams.from_hparams('./hparams/ROME/llama3-8b')
# hparams = ROMEHyperParams.from_hparams('./hparams/ROME/gemma-7b')
# hparams = MEMITHyperParams.from_hparams('./hparams/MEMIT/llama3-8b')

hparams.device = 1
editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=questions,
    # rephrase_prompts=paraphrased_questions,
    target_new=targets,
    subject=subjects,
    # portability_inputs=portability_inputs,
    summary_metrics=True,
    keep_original_weight=True,
    # test_generation=True,
)

json.dump(metrics, open(os.path.join('../results/', f'tmp_ROME_{model_id_format}_results.json'), 'w'), indent=4)
del edited_model
gc.collect()
torch.cuda.empty_cache()

2024-08-12 18:46:29,734 - hallucination_editor - INFO - Instantiating model
08/12/2024 18:46:29 - INFO - hallucination_editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/data1/baixiang/env/anaconda3/envs/edit/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  2%|▏         | 1/50 [00:00<00:33,  1.46it/s]

===== Question: Which tourist attraction was owned by Greece? | Prediction: Knossos. | Label: Parthenon | Evaluation: 0 =====


  4%|▍         | 2/50 [00:01<00:24,  1.92it/s]

...

===== Question: What is the architectural style of Willis Tower? | Prediction: International Style. | Label: International Style | Evaluation: 1 =====
Executing ROME algorithm for the update: [Which tourist attraction's creator is Jan Styka?] -> [ Racławice Panorama]
Computing left vector (u)...
Selected u projection object Jan Styka
Left vector shape: torch.Size([14336])
Computing right vector (v)
Lookup index found: 9 | Sentence: Which tourist attraction's creator is Jan Styka? Racławice Pan | Token: ka
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 2.729 = 2.729 + 0.0 + 0.0 avg prob of [ Racławice Panorama] 0.06627755612134933
loss 2.668 = 2.524 + 0.143 + 0.001 avg prob of [ Racławice Panorama] 0.08082066476345062
loss 2.083 = 2.059 + 0.022 + 0.001 avg prob of [ Racławice Panorama] 0.12921631336212158
loss 1.588 = 1.552 + 0.035 + 0.001 avg prob of [ Racławice Panorama] 0.2130821794271469
loss 3.13 = 3.099 + 0.03 + 0.001 avg prob of [ Racławi

2024-08-12 18:50:12,699 - hallucination_editor - INFO - Execution 49 editing took 2.6805195808410645
08/12/2024 18:50:12 - INFO - hallucination_editor -   Execution 49 editing took 2.6805195808410645


loss 0.045 = 0.033 + 0.011 + 0.001 avg prob of [ Racławice Panorama] 0.9680377840995789
Delta norm: 12.2578125
Change in target norm: 3.064453125 to 12.6328125 => 9.5703125
Division Factor: 3.83984375
Right vector norm: 3.19140625
Right vector shape: torch.Size([4096])
Deltas successfully computed for ['model.layers.5.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.5.mlp.down_proj.weight']


2024-08-12 18:50:13,151 - hallucination_editor - INFO - Evaluation took 0.45148134231567383
08/12/2024 18:50:13 - INFO - hallucination_editor -   Evaluation took 0.45148134231567383
2024-08-12 18:50:13,152 - hallucination_editor - INFO - 49 editing: Which tourist attraction's creator is Jan Styka? -> Racławice Panorama  
 {'pre': {'edit_acc': [0], 'edit_output': ['The National Shrine of the Immaculate Conception.'], 'locality': {}, 'portability': {}, 'yes_no': {}}, 'case_id': 49, 'requested_edit': {'prompt': "Which tourist attraction's creator is Jan Styka?", 'target_new': 'Racławice Panorama', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {}, 'yes_no': {}, 'harm_original_text': {}, 'subject': 'Jan Styka'}, 'time': 2.6805195808410645, 'post': {'edit_acc': [1], 'edit_output': ['Racławice Panorama.'], 'locality': {}, 'portability': {}, 'yes_no': {}}}
08/12/2024 18:50:13 - INFO - hallucination_editor -   49 editing: Which tourist attraction's creator is Jan Styka? -> Rac

===== Question: Which tourist attraction's creator is Jan Styka? | Prediction: Racławice Panorama. | Label: Racławice Panorama | Evaluation: 1 =====
Metrics Summary:  {'pre': {'edit_acc': 0.0}, 'post': {'edit_acc': 0.86}, 'time': 3.556901044845581}
